Follow https://www.freecodecamp.org/news/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04/ 

In [1]:
import re 
import nltk
import pandas as pd
import string

import numpy as np

# nltk.download("stopwords")

from nltk.corpus import stopwords 

In [2]:
df = pd.read_json('gs://cytora-user-wout/News_Category_Dataset_v2.json', lines=True)

### Step 1: Tokenize a sentence 

In [3]:
stopwords_set = set(stopwords.words('english'))

In [4]:
def word_extraction(sentence):
    ignore = list(stopwords_set)    
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w.lower() not in ignore]    
    return cleaned_text

In [5]:
for sen in df['headline'].sample(5):
    print(sen)
    print(word_extraction(sen))
    print()

The Bird Flu Debate: How to Avoid Ruffling Feathers While Still Having a Real Conversation
['bird', 'flu', 'debate', 'avoid', 'ruffling', 'feathers', 'still', 'real', 'conversation']

Seth Rogen Says He Smoked Weed In Steven Spielberg's Face
['seth', 'rogen', 'says', 'smoked', 'weed', 'steven', 'spielberg', 'face']

Clinton Campaign Chairman On Trade Deal: 'Can You Make It Go Away?'
['clinton', 'campaign', 'chairman', 'trade', 'deal', 'make', 'go', 'away']

People Fed Up With ‘Thoughts And Prayers’ Demand Action After Texas Church Massacre
['people', 'fed', 'thoughts', 'prayers', 'demand', 'action', 'texas', 'church', 'massacre']

10 Simple Strategies for a Clean and Organized Home
['10', 'simple', 'strategies', 'clean', 'organized', 'home']



**TODO** Words like `isn't` and `U.S.` aren't extracted properly, maybe in case of `isn't` it doesn't matter as it will just get mapped to `isn`. 

### Step 2: Apply tokenization to all sentences
 

In [6]:
def tokenize(sentences):    
    words = []    
    for sentence in sentences:        
        w = word_extraction(sentence)        
        words.extend(w)            
        words = sorted(list(set(words)))    
    return words

In [7]:
sample2 = df['headline'].sample(2)
display(sample2)
tokenize(sample2)

99884    Dame Helen Mirren Rides The Subway Like The Re...
73752    These Two People Could Not Be More Unimpressed...
Name: headline, dtype: object

['could',
 'dame',
 'hanks',
 'helen',
 'like',
 'mirren',
 'people',
 'plebeians',
 'rapping',
 'rest',
 'rides',
 'subway',
 'tom',
 'two',
 'unimpressed',
 'us']

### Step 3: Build vocabulary and generate vectors

In [8]:
def generate_bow(allsentences):        
    vocab = tokenize(allsentences)
    vector_space = pd.DataFrame()
    print(f'Done with tokenising vocabular of length: {len(vocab)}')

    for sentence in allsentences:
        words = word_extraction(sentence)
        bag_vector = np.zeros(len(vocab))
        for w in words:
            for i, word in enumerate(vocab):
                if word == w:
                    bag_vector[i] += 1
            vector_space[sentence] = np.array(bag_vector)
#     vector_space = 
    return vocab, vector_space.T

**TODO** why is the world `the` still in there? 

In [9]:
# df=df.sample(100) # REMOVE THIS STEP 

In [ ]:
now = pd.datetime.now() 

vocab, vector_space = generate_bow(df['headline'])

diff = pd.datetime.now() - now
print(f'{diff.seconds / 60:.1f} minutes to run')

In [ ]:
vector_space.columns = [str(x) for x in vocab]

In [ ]:
vector_space.to_csv('data/02_bag_of_words.csv')

**TODO:** fix this

In [152]:
# vector_space[
#     (vector_space['04'] == 1) |
#     (vector_space['000'] == 1) |
#     (vector_space['1'] == 1) |
#     (vector_space['102'] == 1) 
# ].head()

In [153]:
vector_space = vector_space.reset_index()

**TODO** maybe remove vectors that represent < 6 words, this will be random anyway 

In [154]:
vector_space.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 581 entries, index to young
dtypes: float64(580), object(1)
memory usage: 454.0+ KB


In [157]:
len(df['category'].values)

100

# CHECK IF STILL NAMED INDEX

Otherwise change name in following cell 


In [161]:
vector_space.head()

,index,000,10,12,15,20,2012,4,4th,5,...,work,workouts,world,worse,would,wow,year,yoga,york,young
0,What You Need To Make a Bangin' Summer Salad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Homemade Ricotta Cheese,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Carl Bernstein: Donald Trump's Disdain For Fac...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bethenny Frankel's Ex Reportedly Says Daughter...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TV Characters In Halloween Costumes: The Super...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
vector_space.rename(columns={'index': 'headline'}, inplace=True)

In [170]:
output = vector_space.merge(df[['category', 'headline']], suffixes=('', '_target'), how='inner', on='headline')

In [ ]:
output.to_csv('data/02_bag_of_words.csv', index=False)